# Start of the analysis notebook

**Author : Inga Ulusoy**  
*Date : 06/25*  
*Affiliation : SSC*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# required modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from pathlib import Path

In [ ]:
# constants and global functions/variables
data_file_path = Path("../data")

In [ ]:
# reading of the data files
def read_pandas(path, name):
    file = path / name
    return pd.read_csv(file, sep=r"\s+")  # noqa


def read_numpy(path, name, skip=1):
    file = path / name
    return np.loadtxt(file, skiprows=skip)

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# read and plot expec.t
data_expec = read_pandas(data_file_path, "expec.t")
data_expec.head()

In [ ]:
data_expec.plot(x="time", y="<z>")
plt.savefig("expec_t.png", dpi=300)
plt.show()

In [ ]:
sn.relplot(data=data_expec, kind="line", x="time", y="<z>")
plt.show()

In [ ]:
sn.pairplot(data_expec, corner=True)
plt.savefig("pairplot_expec_t.png", dpi=300)
plt.show()

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded
data_expec.var()

In [ ]:
thresh = 1.0e-3

In [ ]:
# df_expec.drop(df_expec.var()[df_expec.var()<threshv].index.values, axis=1)
# print(df_expec.var()<threshv) # this will return columns with variance below the threshold
# df_expec.var()[df_expec.var()<threshv].index # this will return the index of those columns
# df_expec.var()[df_expec.var()<threshv].index.values # this will return the index of those columns in an array
# df_expec.drop(df_expec.var()[df_expec.var()<threshv].index.values) # we want to drop those columns
data_expec.drop(
    data_expec.var()[data_expec.var() < thresh].index.values, axis=1
)  # but we have to pick the right axis (columns, not rows)

In [ ]:
def check_if_significant(data_in, thresh):
    data_out = data_in.drop(data_in.var()[data_in.var() < thresh].index.values, axis=1)
    indices = data_in.var()[data_in.var() > thresh].index.values
    return data_out, indices

In [ ]:
df_expec, indices = check_if_significant(data_expec, thresh)  # flake8-noqa-cell
display(df_expec.head())
print(indices)

### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots
# I am leaving this to you

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
# read in npop.t
data_npop = read_pandas(data_file_path, "npop.t")
data_npop.head()

In [ ]:
# flake8-noqa-cell
# discard all columns with variance below a set threshold - we can consider them as constant
df_npop, indices_npop = check_if_significant(data_npop, thresh)
display(df_npop.head())
print(indices_npop)

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
# plot ideally with seaborn
sn.lineplot(x="time", y="value", hue="variable", data=pd.melt(df_npop, ["time"]))
# Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.
plt.savefig("npop_t.png", dpi=300)
plt.show()

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix
print("Correlation Matrix")
print(df_npop.corr())

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# flake8-noqa-cell
# get rid of time column, lower triangular and diagonal entries of the correlation matrix
# sort the remaing values according to their absolute value, but keep the sign
def get_correlation_measure(df):
    drop_values = set()  # an unordered collection of items
    cols = df.columns  # get the column labels
    print(cols)
    for i in range(0, df.shape[1]):
        for j in range(
            0, i + 1
        ):  # get rid of all diagonal entries and the lower triangular
            drop_values.add((cols[i], cols[j]))
    print(drop_values)
    return drop_values


df_npop_short = df_npop.drop(["time"], axis=1)  # get rid of time column
drop_vals = get_correlation_measure(
    df_npop_short
)  # get rid of lower triangular and diagonal entries
corr2 = df_npop_short.corr().unstack()  # pivot the correlation matrix
# sort by absolute values but keep sign
corr2 = corr2.drop(labels=drop_vals).sort_values(
    ascending=False, key=lambda col: col.abs()
)
display(corr2)

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
data_table = read_numpy(data_file_path, "table.dat")
data_table[0]
# replace the NaNs by zero
data_table = np.nan_to_num(data_table)

Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance
def euclidean_distance(list_ref, list_comp, vectors):
    distances = np.zeros(len(list_ref))
    for i in range(len(list_ref)):
        distances[i] = np.linalg.norm(vectors[list_comp[i]] - vectors[list_ref[i]])
    return distances

In [ ]:
# plot the result and save to a .pdf
out_dist = euclidean_distance([2, 4, 6], [3, 5, 7], data_table)
print(out_dist)
x = range(0, len(out_dist))
plt.bar(x, out_dist)
plt.xticks(x, ("x", "y", "z"))
plt.savefig("euclidean_distance.pdf", dpi=300)
plt.show()

In [ ]:
# print the result to a file

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# define some global functions

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# read and plot efield.t
efield = read_numpy(data_file_path, "efield.t", skip=1)
efield = efield.T

Here we are interested in column 2 since the others are constant.

In [ ]:
# discard the columns with variance below threshold - these are considered constant
print(np.var(efield))  # var computed for the overall, flattened array
print(np.var(efield, axis=1))  # var computed for each column (row-major order)
print(
    np.nonzero(np.var(efield, axis=1) > 1e-4)
)  # returns the indices of values for which condition applies
print(
    efield[np.nonzero(np.var(efield, axis=1) > 1e-4)]
)  # get the numpy array for those indices

In [ ]:
def check_if_significant_np(data_in, thresh):
    indices = np.nonzero(np.var(data_in, axis=1) > thresh)
    data_out = data_in[indices]
    return data_out, indices

In [ ]:
# do the FT
def do_DFT(data, tmax):
    data_s = np.fft.rfft(data)
    data_w = np.fft.rfftfreq(tmax)
    return data_s, data_w

In [ ]:
efield2, indices = check_if_significant_np(efield, thresh=1e-4)
print(efield2)
plt.plot(efield2[1])
plt.savefig("efield2.png", dpi=300)
plt.show()

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies

In [ ]:
efield_s, frequency = do_DFT(efield2[1], len(efield2[0]))
plt.plot(frequency, abs(efield_s) ** 2)  # the frequency of this laser pulse was 0.116
plt.savefig("efield_spectrum.png", dpi=300)
plt.show()

### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array
wavef = read_numpy(data_file_path, "nstate_i.t")

In [ ]:
# store the time column (column 0) in a vector and drop from array
time = wavef[0]
wavef = np.delete(wavef, [0], axis=0)
print(wavef[0])

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array
# convert to complex array
realpart = wavef[0::2]
imagpart = wavef[1::2]
wavefc = realpart + 1j * imagpart
print(wavefc[0])
print(realpart[0])
print(imagpart[0])

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step
def calc_auto(wavef):
    aucofu = np.zeros(len(wavef[0]), dtype=complex)
    for i in range(0, len(wavef[0])):
        aucofu[i] = np.sum(wavef[:, 0] * np.conjugate(wavef[:, i]))
    return aucofu

In [ ]:
aucofu = calc_auto(wavefc)
print(aucofu)
plt.plot(abs(aucofu**2))
plt.savefig("autocorrelation_function.png", dpi=300)
plt.show()

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# flake8-noqa-cell
# discrete Fourier-transform the autocorrelation function - now we need all frequency components,
# also the negative ones
# now do the FT
# do the FT - see https://numpy.org/doc/stable/reference/routines.fft.html
def do_fft(data, tmax):
    data_s = np.fft.fft(data)
    data_w = np.fft.fftfreq(tmax)
    # only take the positive frequency components
    return data_w[0 : tmax // 2], data_s[0 : tmax // 2]


energy, spec = do_fft(aucofu, len(time))
print(energy)

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)
# plt.plot(energy,real(spec))
# plt.plot(energy,imag(spec))
# plt.plot(energy,abs(spec))
plt.plot(energy, abs(spec) ** 2)
plt.savefig("autocorrelation_spectrum.png", dpi=300)